In [3]:
import io
import re
import icu
import json

with io.open("data/Unihan_Readings.json", "r", encoding = "utf-8") as r:
    readings = json.loads(r.read())
with io.open("data/Unihan_DictionaryLikeData.json", "r", encoding = "utf-8") as dl:
    dldata = json.loads(dl.read())

# for kCheungBauer that has [ng] or 1/2 etc
def extract(pron):
    output = []
    for p in pron:
        if p.startswith("[ng]"):
            output += extract([p[4:], "ng" + p[4:]])
        elif re.match("[a-z]+[1-6](/[1-6])+", p):
            index = re.findall("[a-z]+", p)[0]
            nums = re.findall("[1-6]", p)
            output += [index + n for n in nums]
        else:
            output.append(p)
    return output
    
def main():
    jyutpingSet = set()
    jyutpingDetonedSet = set()
    for han in readings:
        if "kCantonese" in readings[han]:
            for pron in readings[han]["kCantonese"]:
                detoned = pron[:-1] # remove number 1-6
                jyutpingSet.add(pron)
                jyutpingDetonedSet.add(detoned)
    for han in dldata:
        if "kCheungBauer" in dldata[han]:
            for data in dldata[han]["kCheungBauer"]:
                subdata = data.split(";")
                prons = subdata[2].split(",")
                pronss = extract(prons)
                for pron in pronss:
                    detoned = pron[:-1] # remove number 1-6
                    jyutpingSet.add(pron)
                    jyutpingDetonedSet.add(detoned)
    
    # simple sort is enough
    jyutpings = { "toned" : sorted(jyutpingSet), "detoned" : sorted(jyutpingDetonedSet) }

    with io.open("data/jyutping-index.json", "w", encoding = "utf-8") as j:
        json.dump(jyutpings, j, ensure_ascii = False, separators = (",", ":"))

if __name__ == "__main__":
    main()
